In [1]:
import pandas as pd
import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

In [2]:
cleaned = pd.read_csv('datasets/cleaned_Izends_Data_Thru201712_ver5.csv',index_col=0)

/home/ambuj/izenda_fullcycle/environment/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cleaned_loss_des = pd.read_csv('hashed_cleaned_loss_desc_from_5_ipynb.csv',index_col=0)
# changing dtype of hashed_loss_desc_padded column
cleaned_loss_des['hashed_loss_desc_padded'] = cleaned_loss_des['hashed_loss_desc_padded'].apply(lambda x: np.fromstring(x[1:-1],dtype='int32',sep=' '))

# Changing no of bins from 10 to 5

In [4]:
pd.qcut(cleaned_loss_des['EstimateTotal'],5).value_counts().sort_index(ascending=True)

(-0.0009999999999716, 924.544]    153282
(924.544, 2023.348]               153281
(2023.348, 3864.61]               153283
(3864.61, 8323.578]               153280
(8323.578, 265738.05]             153282
Name: EstimateTotal, dtype: int64

In [5]:
cleaned_loss_des['EstimateTotal_bins'] = pd.qcut(cleaned['EstimateTotal'],5,labels = ["a", "b", "c", "d","e"])

# Merging Data

In [6]:
cleaned_combined = pd.merge(cleaned,cleaned_loss_des[['EstimateTotal_bins','hashed_loss_desc_padded']],left_index=True,right_index=True)

In [7]:
cleaned_combined.shape

(766408, 28)

In [8]:
cleaned_combined.columns

Index(['DisplayName', 'DivisionName', 'City', 'Zip', 'State', 'LossYearMo',
       'StartedFlag', 'CommOrRes', 'NoteCount', 'PhotoCount', 'JobCount',
       'ClaimCount', 'EstimateTotal', 'PolicyHolderType', 'LossDescription',
       'Estimate_NetClaim', 'Estimate_MaterialSaleTax', 'Estimate_OverHead',
       'Estimate_Profit', 'Estimate_PctOverhead', 'Estimate_PctProfit',
       'Estimate_Deductible', 'Estimate_BaseSvcCharge',
       'CleanAddressFranchisorID', 'CleanAddressContactID', 'LossMo',
       'EstimateTotal_bins', 'hashed_loss_desc_padded'],
      dtype='object')

# Defining X and Y

In [9]:
X1 = cleaned_combined[['DisplayName', 'DivisionName','LossMo','CommOrRes','PolicyHolderType','NoteCount','PhotoCount','JobCount','ClaimCount','LossYearMo']]
X2 = cleaned_combined['hashed_loss_desc_padded']
y = cleaned_combined['EstimateTotal_bins']

# Getting Dummies

In [10]:
X1_dummies = pd.get_dummies(X1,sparse=True)

In [11]:
all_data = pd.merge(X1_dummies,pd.DataFrame(X2),left_index=True,right_index=True)
all_data['EstimateTotal_bins'] = y

# Splitting data

In [12]:
test2017=all_data[all_data["LossYearMo"].apply(lambda all_data:all_data>=201701)]
learning=all_data[all_data["LossYearMo"].apply(lambda all_data:all_data<201701)]

In [13]:
learning_x = learning[learning.columns.difference(['LossYearMo','EstimateTotal_bins'])]
learning_y = learning['EstimateTotal_bins']
learning_y_dummies = pd.get_dummies(learning_y,sparse=True)

In [14]:
test2017_x1 = test2017[test2017.columns.difference(['LossYearMo','EstimateTotal_bins','hashed_loss_desc_padded'])]
test2017_x2 = test2017['hashed_loss_desc_padded']
test2017_y = test2017['EstimateTotal_bins']
test2017_y_dummies = pd.get_dummies(test2017_y,sparse=True)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(learning_x, learning_y_dummies, test_size=0.33, random_state=42)

In [16]:
X1_train = X_train[X_train.columns.difference(['hashed_loss_desc_padded'])]
X2_train = X_train['hashed_loss_desc_padded']

In [17]:
X1_val = X_val[X_val.columns.difference(['hashed_loss_desc_padded'])]
X2_val = X_val['hashed_loss_desc_padded']

# Building Custom Keras Combined Model using Functional API

In [18]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, History 
from keras.layers.merge import concatenate

Using TensorFlow backend.


In [19]:
# first input
input1 = Input(shape=(X1_train.shape[1],),name='input1')
hidden1 = Dense(512, activation='relu')(input1)
dropout1 = Dropout(0.25)(hidden1)
hidden2 = Dense(512, activation='relu')(dropout1)
dropout2 = Dropout(0.25)(hidden2)

In [20]:
# second input
input2 = Input(shape=(14,),name='input2')
embedding = Embedding(11024,100,input_length=14)(input2)
lstm = LSTM(15)(embedding)

In [21]:
# merge layers
merge = concatenate([dropout2,lstm])

In [22]:
# output layer
output = Dense(y_train.shape[1], activation='softmax',name='output')(merge)

In [23]:
# Model
model = Model(inputs=[input1,input2], outputs=output)

In [24]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             (None, 1064)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512)          545280      input1[0][0]                     
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 512)          0           dense_1[0][0]                    
__________________________________________________________________________________________________
input2 (InputLayer)             (None, 14)           0                                            
__________________________________________________________________________________________________
dense_2 (D

In [25]:
plot_model(model,to_file='combined_model.png')

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
early_stop = EarlyStopping(monitor='val_loss', patience=50,restore_best_weights=True)

In [28]:
model.fit({'input1':X1_train,'input2':np.stack(X2_train, axis=0)},
          {'output':y_train.values},
          validation_data=({'input1':X1_val,'input2':np.stack(X2_val, axis=0)},{'output':y_val.values}),
         callbacks=[early_stop],epochs=500, batch_size=128,verbose=1)

Train on 335176 samples, validate on 165087 samples
Epoch 1/500
335176/335176 [==============================] - 81s 242us/step - loss: 1.4023 - acc: 0.3773 - val_loss: 1.3533 - val_acc: 0.4006
Epoch 2/500
335176/335176 [==============================] - 80s 239us/step - loss: 1.3395 - acc: 0.4092 - val_loss: 1.3466 - val_acc: 0.4053
Epoch 3/500
335176/335176 [==============================] - 80s 238us/step - loss: 1.3158 - acc: 0.4234 - val_loss: 1.3452 - val_acc: 0.4038
Epoch 4/500
335176/335176 [==============================] - 80s 239us/step - loss: 1.2948 - acc: 0.4368 - val_loss: 1.3475 - val_acc: 0.4058
Epoch 5/500
335176/335176 [==============================] - 80s 238us/step - loss: 1.2730 - acc: 0.4481 - val_loss: 1.3599 - val_acc: 0.4034
Epoch 6/500
335176/335176 [==============================] - 80s 238us/step - loss: 1.2512 - acc: 0.4605 - val_loss: 1.3751 - val_acc: 0.4001
Epoch 7/500
335176/335176 [==============================] - 80s 238us/step - loss: 1.2301 - acc

# Calculating Test Accuracy

In [29]:
test2017_preds_proba = model.predict({'input1':test2017_x1,'input2':np.stack(test2017_x2, axis=0)})

In [30]:
test2017_preds = np.argmax(test2017_preds_proba,axis=1)

In [31]:
mapping_dict = {0:'a',1:'b',2:'c',3:'d',4:'e'}

In [32]:
test2017_preds_names = pd.Series(test2017_preds).map(mapping_dict)

In [33]:
from sklearn.metrics import accuracy_score

In [34]:
accuracy_score(test2017_y,test2017_preds_names)

0.40343797553964944